### Execução do algoritmo Apriori - Biblioteca mlxtend
http://rasbt.github.io/mlxtend/

# Bibliotecas

In [195]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from decimal import Decimal

# Carregamento dos Dados - .csv

In [196]:
anoArq = '2018'

In [197]:
# endereco = '../BaseDadosSelecionados/'
# endereco2 = 'Agrupado_Ocorrencia'

In [198]:
nomeArq = 'ES_datatran' + anoArq +'_Org_Apriori'

In [199]:
endereco= 'PorAno/'
# endereco= 'PorData/'

In [200]:
data = pd.read_csv(endereco + nomeArq + '.csv', encoding='ISO-8859-1', sep=';', header=None)

In [201]:
data.shape

(2640, 10)

In [202]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9
0,dia_semana=Segunda-feira,causa_acidente=Ultrapassagem Indevida,tipo_acidente=Colisão transversal,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Crescente,condicao_metereologica=Nublado,tipo_pista=Simples,tracado_via=Reta,uso_solo=Não
1,dia_semana=Segunda-feira,causa_acidente=Falta de Atenção à Condução,tipo_acidente=Colisão traseira,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Decrescente,condicao_metereologica=Nublado,tipo_pista=Simples,tracado_via=Reta,uso_solo=Sim
2,dia_semana=Segunda-feira,causa_acidente=Velocidade Incompatível,tipo_acidente=Saída de leito carroçável,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Crescente,condicao_metereologica=Ignorado,tipo_pista=Simples,tracado_via=Não Informado,uso_solo=Não
3,dia_semana=Segunda-feira,causa_acidente=Fenômenos da Natureza,tipo_acidente=Queda de ocupante de veículo,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Crescente,condicao_metereologica=Chuva,tipo_pista=Simples,tracado_via=Reta,uso_solo=Não
4,dia_semana=Terça-feira,causa_acidente=Falta de Atenção do Pedestre,tipo_acidente=Atropelamento de Pedestre,classificacao_acidente=Com Vítimas Feridas,fase_dia=Plena Noite,sentido_via=Decrescente,condicao_metereologica=Céu Claro,tipo_pista=Dupla,tracado_via=Reta,uso_solo=Sim


# Carregamento dos dados em uma matriz (Importante)

In [203]:
%%time
# data.shape[0] LINHAS e data.shape[1] COLUNAS

dataset = []
for i in range(0, data.shape[0]):
    dataset.append([str(data.values[i,j]) for j in range(0, data.shape[1])])

Wall time: 324 ms


In [204]:
# dataset

# Tratamento para a execução do algoritmo Apriori

http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/

In [205]:
%%time
te = TransactionEncoder()

Wall time: 0 ns


## 1º Transformação - Representação para uma matriz de boleanos

In [206]:
%%time
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

Wall time: 19 ms


In [207]:
# df

## 2ª Transformação - Representação esparsa
Exemplo 3 - Trabalhando com representações esparsas - http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/ 

In [208]:
# %%time
# oht_ary = te.fit(dataset).transform(dataset, sparse=True)
# sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)

In [209]:
# sparse_df

# Execução do algoritmo apriori

In [210]:
numMinSupport = 0.1

In [211]:
%%time
# sparse_df
frequent_itemsets = apriori(df, min_support=numMinSupport, use_colnames=True)

Wall time: 57 ms


In [212]:
# frequent_itemsets

In [213]:
# metric = lift

In [214]:
%%time
association_rules_saida = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
# association_rules_saida = association_rules(frequent_itemsets)

Wall time: 17 ms


In [215]:
saidaRecords = pd.DataFrame(association_rules_saida)

## Método para contabilizaro número de consequentes e antecedentes em cada regra

In [216]:
# Cria duas novas colunas contabilizando os antecedentes e consequents
saidaRecords["antecedent_len"] = saidaRecords["antecedents"].apply(lambda x: len(x))
saidaRecords["consequent_len"] = saidaRecords["consequents"].apply(lambda x: len(x))

## Filtro

In [217]:
# saidaRecords = saidaRecords[ (saidaRecords['antecedent_len'] >= 2 ) & (saidaRecords['consequent_len'] >= 1) ] # & (saidaRecords['support'] >= 0.5) 

In [218]:
# saidaRecords = saidaRecords[(saidaRecords['support'] >= 0.6)]

In [219]:
saidaRecords.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,(causa_acidente=Desobediência às normas de trâ...,(classificacao_acidente=Com Vítimas Feridas),0.142803,0.831818,0.125379,0.877984,1.055500,0.006593,1.378360,1,1
1,(causa_acidente=Falta de Atenção à Condução),(classificacao_acidente=Com Vítimas Feridas),0.419697,0.831818,0.368182,0.877256,1.054625,0.019070,1.370187,1,1
2,(condicao_metereologica=Céu Claro),(classificacao_acidente=Com Vítimas Feridas),0.546970,0.831818,0.459470,0.840028,1.009869,0.004490,1.051318,1,1
3,(condicao_metereologica=Nublado),(classificacao_acidente=Com Vítimas Feridas),0.207197,0.831818,0.171212,0.826325,0.993397,-0.001138,0.968373,1,1
4,(dia_semana=Quarta-feira),(classificacao_acidente=Com Vítimas Feridas),0.124242,0.831818,0.101136,0.814024,0.978609,-0.002211,0.904322,1,1


In [220]:
saidaRecords.max()

antecedents           (tracado_via=Reta, uso_solo=Não, sentido_via=C...
consequents                                        (tipo_pista=Simples)
antecedent support                                             0.595455
consequent support                                             0.831818
support                                                        0.497727
confidence                                                     0.920128
lift                                                            1.92916
leverage                                                       0.131474
conviction                                                      5.59606
antecedent_len                                                        3
consequent_len                                                        2
dtype: object

## Salvando os resultados do algoritmo em um .csv

In [221]:
saidaRecords.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,(causa_acidente=Desobediência às normas de trâ...,(classificacao_acidente=Com Vítimas Feridas),0.142803,0.831818,0.125379,0.877984,1.055500,0.006593,1.378360,1,1
1,(causa_acidente=Falta de Atenção à Condução),(classificacao_acidente=Com Vítimas Feridas),0.419697,0.831818,0.368182,0.877256,1.054625,0.019070,1.370187,1,1
2,(condicao_metereologica=Céu Claro),(classificacao_acidente=Com Vítimas Feridas),0.546970,0.831818,0.459470,0.840028,1.009869,0.004490,1.051318,1,1
3,(condicao_metereologica=Nublado),(classificacao_acidente=Com Vítimas Feridas),0.207197,0.831818,0.171212,0.826325,0.993397,-0.001138,0.968373,1,1
4,(dia_semana=Quarta-feira),(classificacao_acidente=Com Vítimas Feridas),0.124242,0.831818,0.101136,0.814024,0.978609,-0.002211,0.904322,1,1


## Substituição de '.' para ','

In [222]:
listCol = ['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction']
for i in listCol:
    saidaRecords[i] = saidaRecords[i].astype(str)
    saidaRecords[i] = saidaRecords[i].str.replace('.', ',')

## Ordena os dados baseado na confiança

In [223]:
# Ordena pela saidaRecords e salva em um novo DataFrame
saidaRecords = saidaRecords.sort_values(by='confidence', ascending=False)

## Salva os resultados em um arquivo .csv

In [224]:
# Salvando em um arquivo .csv
saidaRecords.to_csv(endereco + 'AprioriMlxtend_'+ 'ES_datatran' + anoArq + '.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

In [225]:
# Gambiarra para atualizar o arquivo com os dados ordenados, de acordo com a célula anterior
# saidaRecords = pd.read_csv(endereco + 'AprioriMlxtend_'+ 'ES_datatran' + anoArq + '.csv', encoding='ISO-8859-1', sep=';')

In [226]:
# Salvando em um arquivo .csv
# saidaRecords.to_csv(endereco + 'AprioriMlxtend_'+ 'ES_datatran' + anoArq + '.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

### Filtro para facilitar a visualização dos resultados

In [227]:
# rules[ (rules['antecedent_len'] >= 2) &
#        (rules['confidence'] > 0.75) &
#        (rules['lift'] > 1.2) ]

In [228]:
# rules[rules['antecedents'] == {'Eggs', 'Kidney Beans'}]

## Filtros modelos para serem utilizados no apriori

In [229]:
# saidaRecords[ (saidaRecords['antecedent_len'] >= 3 ) &
#                    (saidaRecords['consequent_len'] >= 3) ]

In [230]:
# frequent_itemsets[ (frequent_itemsets['length'] == 2) &
#                    (frequent_itemsets['support'] >= 0.8) ]

In [231]:
# frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]

# ATIVIDADES FUTURAS

- Verificar se a coluna ['ignorados'] é necessária para analisar os resultados. <br>
Analisar o significado da coluna 'ignorados' no site da PRF. Ou no arquivo de descrição de tabelas.
***Resposta** : No link do anuário 2020, lá é apresentado alguns detalhes dos dados.

- Estudar os parâmetros do algoritmo apriori.

**[Resolvido]**
- Analisar a saída do 'lift' e 'conviction'. 
***Resposta**: O problema está no momento que os dados 'lift' e 'conviction' são passados para o arquivo .csv.

No Jupyter os dados estão sendo apresentados da maneira correta.

# Print Final

In [232]:
print("TUDO EXECUTADO!")

TUDO EXECUTADO!
